# Example with Simulated Data in non-BIDS format

The point of this notebook is to show how to set up the data without BIDS.
We start by loading the important packages.

In [1]:
import os

import numpy as np

from py_neuromodulation import (
    nm_analysis,
    nm_define_nmchannels,
    nm_IO,
    nm_stream_offline,
    nm_plots
)

import matplotlib.pyplot as plt
# Reload imports to get changes in other scripts
%load_ext autoreload
%autoreload 2

Set up the necessary paths to the data.

In [2]:
SCRIPT_PATH = os.path.dirname(os.path.abspath(''))
DATA_PATH = os.path.join(SCRIPT_PATH,'examples','data','simulated_Data','data.mat')

PATH_OUT = os.path.join(SCRIPT_PATH,'examples','data','simulated_Data')

__Here we load our data.__

The data is saved in examples/data/simulated_Data/data.mat

From nm_IO we use the loadmat function to load this data, and then we can see that the channel names, as well as the bad channels and the sampling frequency are also specified.

The scrip that generates the data is examples/createSimulatedData.py

In [3]:
data_dict = nm_IO.loadmat(DATA_PATH)

In [4]:
data = data_dict['data'] # voltage values for our simulated data
channels = data_dict['channels'] # array with all channel names
bad = data_dict['bad'] # bad channels
sfreq = data_dict['sfreq']  # sampling frequency

In order to give the channel names to the Stream class, we need them to be set with
```nm_define_nmchannels.set_channels```.
We also set the type to be "ecog", but please remember this is actually simulated data.

In [5]:
ch_names = list(channels)
ch_types = ["ecog" for _ in range(len(ch_names))]

nm_channels = nm_define_nmchannels.set_channels(
    ch_names=ch_names,
    ch_types=ch_types,
    reference="default",
    bads=None,
    new_names="default",
    used_types=["ecog"],
)

stream = nm_stream_offline.Stream(
    settings=None,
    nm_channels=nm_channels,
    verbose=False,          # Change here if you want to see the outputs of the run
)

stream.set_settings_fast_compute()
stream.settings['features']['raw_hjorth'] = True
stream.settings['features']['bandpass_filter'] = False
stream.settings['features']['fft'] = True
stream.settings['features']['sharpwave_analysis'] = False
stream.settings['postprocessing']['feature_normalization'] = False
stream.settings['feature_normalization_settings']['normalization_time_s'] = 10
stream.settings['feature_normalization_settings']['normalization_method'] = {
            "mean": False,
            "median": False,
            "zscore": False,
            "zscore-median": True,
            "quantile": False,
            "power": False,
            "robust": False,
            "minmax": False
        }

stream.init_stream(
    sfreq=sfreq,
    line_noise=50,
)

No Error occurred when testing the settings.
No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 7.50 Hz
- Upper transition bandwidth: 7.50 Hz
- Filter length: 999 samples (0.999 sec)



In [6]:
stream.settings; # remove the ; to print out the settings

## Feature Estimation
The feature estimation step is, then, as straightforward as in example_BIDS.ipynb.

In [7]:
stream.run(
    data=data,
    folder_name="simulated_Data_results",
    out_path_root=PATH_OUT,
)

_SIDECAR.json saved to /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/simulated_Data/simulated_Data_results/simulated_Data_results_SIDECAR.json
FEATURES.csv saved to /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/simulated_Data/simulated_Data_results/simulated_Data_results_FEATURES.csv
settings.json saved to /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/simulated_Data/simulated_Data_results/simulated_Data_results_SETTINGS.json
nm_channels.csv saved to /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/simulated_Data/simulated_Data_results/simulated_Data_results_nm_channels.csv
